# Evaluación de PC-SMOTE con Grid Search en el dataset Shuttle (Generación de caso base y datasets aumentados)


In [10]:
# lo que hace es modificar la lista de rutas de búsqueda de módulos de Python (sys.path) para incluir las carpetas ../scripts y ../datasets como ubicaciones adicionales donde Python puede buscar módulos o paquetes cuando hacés un import.
import sys
sys.path.append("../scripts")
sys.path.append("../datasets")

## Importación de módulos y librerías necesarias


In [11]:
# --- Módulos propios del proyecto ---
from cargar_dataset import cargar_dataset, obtener_metadata_dataset # Función para cargar datasets según configuración
from config_datasets import config_datasets                         # Diccionario de configuración de datasets
from evaluacion import evaluar_sampler_holdout                      # Evaluación de sobremuestreo con partición hold-out
from pc_smote import PCSMOTE                                        # Implementación principal de PCSMOTE
from graficador2d import Graficador2D                               # Clase para graficar resultados en 2D
from isolation_cleaner import IsolationCleaner                      # Clase para limpieza de outliers con Isolation Forest
from Utils import Utils                                             # Clase utilitaria con funciones auxiliares
from limpiador import LimpiadorOutliers                             # Clase para limpieza de datos

from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN

# --- Librerías estándar de Python ---
from datetime import datetime, timedelta                       # Manejo de fechas y tiempos
from itertools import product                                  # Generación de combinaciones de parámetros
import os                                                      # Operaciones con el sistema de archivos
from pathlib import Path

import traceback
# --- Librerías científicas ---
import numpy as np                                              # Operaciones numéricas y algebra lineal
import pandas as pd                                             # Manipulación y análisis de datos tabulares
from scipy.stats import uniform                                 # Distribuciones para búsqueda de hiperparámetros

# --- Scikit-learn: preprocesamiento ---
from sklearn.preprocessing import LabelEncoder, StandardScaler # Codificación de etiquetas y escalado de datos
from sklearn.pipeline import make_pipeline, Pipeline            # Creación de pipelines de procesamiento y modelado
from sklearn.preprocessing import RobustScaler

# --- Scikit-learn: división y validación ---
from sklearn.model_selection import (
    train_test_split,                                           # División de datos en train/test
    StratifiedKFold,                                            # Validación cruzada estratificada
    RandomizedSearchCV                                          # Búsqueda aleatoria de hiperparámetros
)

# --- Scikit-learn: reducción de dimensionalidad ---
from sklearn.decomposition import PCA                           # Análisis de Componentes Principales

# --- Scikit-learn: métricas ---
from sklearn.metrics import (
    f1_score,                                                    # Métrica F1-Score
    balanced_accuracy_score,                                     # Precisión balanceada
    matthews_corrcoef,                                           # Coeficiente MCC
    cohen_kappa_score,                                           # Kappa de Cohen
    make_scorer                                            
)

# --- Scikit-learn: clasificadores ---
from sklearn.ensemble import RandomForestClassifier             # Clasificador Random Forest
from sklearn.linear_model import LogisticRegression             # Regresión logística
from sklearn.svm import SVC                                      # Máquinas de Vectores de Soporte (SVM)

from sklearn.exceptions import ConvergenceWarning
import warnings

RANDOM_STATE = 42
RUTA_CLASICOS = "../datasets/datasets_aumentados/resampler_clasicos/"
RUTA_CLASICOS = Path(RUTA_CLASICOS)

## Generación del caso base

Este código realiza dos tareas principales para cada dataset configurado en `config_datasets`:

1. **Generar el caso base** (subcarpeta `datasets_aumentados/base/`):
   - Se crea un directorio específico para almacenar la versión original del dataset sin ningún tipo de sobremuestreo.
   - El dataset se carga utilizando la misma función `cargar_dataset` empleada en el pipeline principal.
   - Si las etiquetas (`y`) están en formato de texto u objeto, se convierten a valores numéricos con `LabelEncoder`.
   - Se realiza una división estratificada en conjuntos de entrenamiento y prueba (`train/test`) utilizando `train_test_split` con una proporción 70/30 y una semilla fija para asegurar reproducibilidad.
   - Se guardan dos archivos CSV: `<nombre_dataset>_train.csv` y `<nombre_dataset>_test.csv`.

In [12]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler

def generar_caso_base(
    nombre_dataset: str,
    config: dict,
    ruta_base: str = "../datasets/datasets_aumentados/base/",
    test_size: float = 0.2,
    random_state: int = 42,
    overwrite: bool = False,
    porcentaje_limpieza: float = 0.0,
):
    """
    Genera el caso base (sin sobremuestreo) aplicando:
      1) Carga del dataset según config_datasets y cargar_dataset()
      2) Split train/test
      3) RobustScaler (fit SOLO en X_train, transform en X_train/X_test)
      4) (Opcional) Limpieza de outliers con IsolationCleaner (IsolationForest por percentil)
         SOLO sobre X_train_scaled.

    IMPORTANTE:
    - `porcentaje_limpieza` se usa para LIMPIAR realmente el conjunto
      de entrenamiento con IsolationCleaner (si > 0). Internamente se pasa como
      `percentil_umbral`.
    - El valor de `porcentaje_limpieza` se refleja en el nombre del archivo de TRAIN
      como sufijo `_I{porcentaje_limpieza}_tm{n_train}` para dejar traza.
    - El TEST nunca se limpia con IsolationForest, por eso su nombre NO lleva sufijo `I*`,
      solo `_tm{n_test}`.

    El resultado son dos CSV:
      - {ruta_base}/{nombre_dataset}_I{porcentaje_limpieza}_tm{n_train}_train.csv
        (train escalado y, si corresponde, limpiado con IF por percentil)
      - {ruta_base}/{nombre_dataset}_tm{n_test}_test.csv
        (test escalado, sin limpieza IF)

    Además, devuelve también:
      - X_test_base, y_test_base (versiones numpy ya escaladas)
    """

    os.makedirs(ruta_base, exist_ok=True)

    col_target = config["col_target"]
    col_features = config.get("col_features", None)

    # Normalizamos el valor de I para el nombre (entero)
    if porcentaje_limpieza is None:
        valor_I = 0
    else:
        valor_I = int(porcentaje_limpieza)

    # ------------------------------------------------------------------
    # CASO 1: intentar reutilizar archivos existentes (patrón con tm)
    # ------------------------------------------------------------------
    path_train_existente = None
    path_test_existente = None

    prefijo_train = f"{nombre_dataset}_I{valor_I}_tm"
    prefijo_test = f"{nombre_dataset}_tm"

    for fname in os.listdir(ruta_base):
        # buscamos algo como: {nombre_dataset}_I{I}_tm{N}_train.csv
        if fname.startswith(prefijo_train) and fname.endswith("_train.csv"):
            path_train_existente = os.path.join(ruta_base, fname)
        # y algo como: {nombre_dataset}_tm{M}_test.csv
        if fname.startswith(prefijo_test) and fname.endswith("_test.csv"):
            path_test_existente = os.path.join(ruta_base, fname)

    if (
        not overwrite
        and path_train_existente is not None
        and path_test_existente is not None
    ):
        print(f"⚠️ Caso base ya existe para {nombre_dataset}. Usando archivos existentes.")
        print(f"   Train existente: {path_train_existente}")
        print(f"   Test existente : {path_test_existente}")

        df_train_existente = pd.read_csv(path_train_existente)
        df_test_existente = pd.read_csv(path_test_existente)

        if col_target not in df_train_existente.columns:
            raise ValueError(
                f"La columna target '{col_target}' no está en el train existente {path_train_existente}"
            )
        if col_target not in df_test_existente.columns:
            raise ValueError(
                f"La columna target '{col_target}' no está en el test existente {path_test_existente}"
            )

        if col_features is None:
            col_features = [c for c in df_train_existente.columns if c != col_target]

        X_test_base = df_test_existente[col_features].values
        y_test_base = df_test_existente[col_target].values

        return path_train_existente, path_test_existente, X_test_base, y_test_base

    # ------------------------------------------------------------------
    # CASO 2: generar caso base desde cero
    # ------------------------------------------------------------------

    # 1) Cargar dataset crudo según config_datasets usando cargar_dataset()
    df_features, y, clases = cargar_dataset(
        path=config["path"],
        clase_minoria=config.get("clase_minoria"),
        col_features=config.get("col_features"),
        col_target=config.get("col_target"),
        sep=config.get("sep"),
        header=config.get("header"),
        binarizar=config.get("binarizar", False),
        tipo=config.get("tipo", "tabular"),
        impute=config.get("impute", "median"),
        na_values=config.get("na_values", ("?", "NA", "None")),
        dataset_name=config.get("dataset_name", nombre_dataset),
        names=config.get("esquema"),
    )

    # X son directamente las features que devuelve cargar_dataset
    X = df_features.values  # ya tiene col_features como columnas

    idx_original = np.arange(len(X), dtype=int)

    tamanio_tn_X = X.shape[0]

    # 2) Train / test split (estratificado)
    X_train, X_test, y_train, y_test, idx_train, idx_test = train_test_split(
        X,
        y,
        idx_original,
        test_size=test_size,
        random_state=random_state,
        stratify=y,
    )


    print(f"[{nombre_dataset}] Split: X_train={X_train.shape}, X_test={X_test.shape}")

    # 3) Escalado robusto (fit en train, transform en train y test)
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # 4) (Opcional) Limpieza de outliers con IsolationCleaner SOLO sobre TRAIN
    if porcentaje_limpieza is not None and porcentaje_limpieza > 0:
        print(
            f"[{nombre_dataset}] Aplicando IsolationCleaner "
            f"(percentil={porcentaje_limpieza}%) sobre TRAIN"
        )

        X_train_scaled, y_train, idx_train, info_limpieza = IsolationCleaner.limpiarOutliers(
            X=X_train_scaled,
            y=y_train,
            idx_original=idx_train,
            percentil_umbral=float(porcentaje_limpieza),
            contamination="auto",
            n_estimators=200,
            max_samples="auto",
            random_state=random_state,
            bootstrap=False,
            normalizar_scores=False,
            devolver_info=True,
            verbose=True,
        )

        removed_total = info_limpieza.get("removed_total", 0)
        total_final = len(y_train)
        total_inicial = removed_total + total_final
        print(
            f"[{nombre_dataset}] Limpieza IF (percentil): "
            f"removidos={removed_total} / total_inicial≈{total_inicial} "
            f"(train_final={total_final})"
        )

    # Si col_features es None acá, las sacamos de df_features:
    if col_features is None:
        col_features = list(df_features.columns)

    # 5) Reconstruir DataFrames con columnas
    df_train = pd.concat(
        [
            pd.DataFrame(X_train_scaled, columns=col_features),
            pd.Series(y_train, name=col_target),
        ],
        axis=1,
    )

    df_test = pd.concat(
        [
            pd.DataFrame(X_test_scaled, columns=col_features),
            pd.Series(y_test, name=col_target),
        ],
        axis=1,
    )

    # Cantidades finales (ya con posible limpieza IF en train)
    n_train_final = df_train.shape[0]
    n_test_final = df_test.shape[0]

    # 6) Construir nombres de archivo con patrón solicitado
    nombre_train = f"{nombre_dataset}_I{valor_I}_tm{n_train_final}_train.csv"
    nombre_test = f"{nombre_dataset}_tdataset{tamanio_tn_X}_tm{n_test_final}_test.csv"

    path_train = os.path.join(ruta_base, nombre_train)
    path_test = os.path.join(ruta_base, nombre_test)

    # 7) Guardar CSV base
    df_train.to_csv(path_train, index=False)
    df_test.to_csv(path_test, index=False)

    print(f"✅ Caso base generado para {nombre_dataset}")
    print(f"   Train: {path_train}  (rows={n_train_final})")
    print(f"   Test : {path_test}   (rows={n_test_final})")

    # 8) Devolver también X_test_base / y_test_base para metadata / métricas
    X_test_base = X_test_scaled
    y_test_base = y_test

    return path_train, path_test, X_test_base, y_test_base, idx_train


In [13]:
def aumentar_dataset_pcsmote_y_guardar( 
    nombre_dataset,
    X_train_base,
    y_train_base,
    percentil_radio_densidad,
    umbral_densidad,
    umbral_pureza_proporcion,
    umbral_riesgo,
    percentil_riesgo,
    criterio_pureza,
    percentil_entropia,
    col_target="target",
    percentil_isolation_etiqueta: float = 0.0,
    X_test_base=None,
    y_test_base=None,
    ruta_pcsmote="../datasets/datasets_aumentados/",
    overwrite=False,
    idx_train=None
):
    try:
        # ================================
        # 1) Crear sampler y resamplear
        # ================================
        sampler = PCSMOTE(
            random_state=42,
            criterio_pureza=criterio_pureza,
            percentil_dist_densidad=percentil_radio_densidad,
            percentil_dist_riesgo=percentil_riesgo,
            percentil_entropia=percentil_entropia,
            umbral_densidad=umbral_densidad,
            umbral_pureza=umbral_pureza_proporcion,
            umbral_riesgo=umbral_riesgo,
            grado_iso=percentil_isolation_etiqueta
        )

        X_res, y_res = sampler.fit_resample(X_train_base, y_train_base, idx_original=idx_train)

        # cantidad de sintéticos generados
        n_original = len(X_train_base)
        n_res = len(X_res)
        cant_sinteticos_generados = n_res - n_original

        # ================================
        # 2) Guardar dataset aumentado
        # ================================
        os.makedirs(ruta_pcsmote, exist_ok=True)

        # ----------------------------
        # Tags para el nombre del file
        # ----------------------------
        # PRD: percentil radio distancia
        tag_prd = f"PRD{int(percentil_radio_densidad)}"

        # PR: percentil riesgo
        tag_pr = f"PR{int(percentil_riesgo)}"

        # CP: criterio pureza (entropía | proporción)
        criterio_lower = str(criterio_pureza).lower()
        if criterio_lower == "entropia":
            tag_cp = "CPent"
        elif criterio_lower == "proporcion":
            tag_cp = "CPprop"
        else:
            raise ValueError(f"Criterio de pureza desconocido: {criterio_pureza}")

        # UD: umbral densidad en %, padded a 3 dígitos → UD080
        valor_ud = int(round(umbral_densidad * 100))
        tag_ud = f"UD{valor_ud:03d}"

        # Pureza extra:
        # - entropía: PE{percentil_entropia} → PE45
        # - proporción: Ppp{upp% en 3 dígitos} → Ppp041
        if criterio_lower == "entropia":
            if percentil_entropia is None:
                raise ValueError("percentil_entropia no puede ser None cuando criterio_pureza='entropia'")
            tag_pureza = f"PE{int(percentil_entropia)}"
        else:  # "proporcion"
            if umbral_pureza_proporcion is None:
                raise ValueError("umbral_pureza_proporcion no puede ser None cuando criterio_pureza='proporcion'")
            valor_upp = int(round(umbral_pureza_proporcion * 100))
            tag_pureza = f"Upp{valor_upp:03d}"

        # UR: umbral riesgo
        tag_ur = f"UR{int(round(umbral_riesgo*100)):03d}"

        # I: percentil de isolation
        tag_iso = f"I{int(percentil_isolation_etiqueta)}"

        # ✅ SV: cantidad de semillas válidas (candidatas)
        semillas_validas = getattr(sampler, "cantidad_semillas_candidatas", 0)
        tag_sv = f"SV{int(semillas_validas):03d}"

        # SG: cantidad de sintéticos generados, padded a 3 dígitos
        tag_sg = f"SG{cant_sinteticos_generados:03d}"

        # Nombre final del archivo
        # Ej entropía:
        #   pcs_ecoli_PRD35_PR35_CPent_UD080_PE45_I0_SG120.csv
        # Ej proporción:
        #   pcs_ecoli_PRD35_PR35_CPprop_UD080_Ppp041_I0_SG007.csv
        nombre_archivo = (
            f"pcs_{nombre_dataset}_"
            f"{tag_prd}_"
            f"{tag_pr}_"
            f"{tag_cp}_"
            f"{tag_ud}_"
            f"{tag_pureza}_"
            f"{tag_ur}_"
            f"{tag_iso}_"
            f"{tag_sv}_"
            f"{tag_sg}_train.csv"
        )

        nombre_archivo = Utils.safe_token(nombre_archivo)
        path_salida = os.path.join(ruta_pcsmote, nombre_archivo)

        if overwrite or not os.path.exists(path_salida):
            # Reconstruir columnas correctamente
            df_res = pd.DataFrame(X_res)
            df_res[col_target] = y_res
            df_res.to_csv(path_salida, index=False)
            print(
                f"   🟢 PCSMOTE guardado: {path_salida} "
                f"(sg={cant_sinteticos_generados})"
            )
        else:
            print(f"   ⚪ Omitido (ya existe): {path_salida}")

        # ================================
        # 3) LOG POR MUESTRA
        # ================================
        df_log = pd.DataFrame(sampler.logs_por_muestra)

        # ================================
        # 4) METADATA DEL DATASET
        # ================================
        metadata = obtener_metadata_dataset(
            nombre_dataset,
            X_train_base,
            y_train_base,
            X_test=X_test_base,
            y_test=y_test_base
        )

        df_header = (
            pd.DataFrame(metadata, index=[0])
            .T.rename(columns={0: "valor"})
        )
        df_header.reset_index(inplace=True)
        df_header.columns = ["campo", "valor"]

        return df_header, df_log, sampler

    except Exception as e:
        traceback.print_exc()
        print(f"❌ Error al aumentar dataset {nombre_dataset}: {e}")
        return None, None, None


In [14]:
def generar_aumentaciones_clasicas_y_guardar(
    nombre_dataset: str,
    X_train: pd.DataFrame,
    y_train: pd.Series,
    col_target: str,
    ruta_clasicos: str = "../datasets/datasets_aumentados/resampler_clasicos/",
    overwrite: bool = False,
    percentil_isolation_etiqueta: float = 0.0
):
    """
    Genera datasets aumentados con técnicas clásicas a partir de X_train, y_train:
      - SMOTE
      - BorderlineSMOTE
      - ADASYN

    Guarda los resultados con patrón:
      {resampler}_{dataset}_I{percentil}_sg{cant_sinteticos}_train.csv
    """

    os.makedirs(ruta_clasicos, exist_ok=True)

    columnas = list(X_train.columns)
    n_original = len(X_train)

    print(f"🔧 Aumentación clásica (en memoria) para: {nombre_dataset}")
    print(f"   X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")

    resamplers = [
        ("smote", SMOTE(random_state=RANDOM_STATE)),
        ("borderlinesmote", BorderlineSMOTE(random_state=RANDOM_STATE, kind="borderline-1")),
        ("adasyn", ADASYN(random_state=RANDOM_STATE)),
    ]

    for nombre_resampler, resampler in resamplers:

        print(f"   🔁 Aplicando {nombre_resampler} ...")

        try:
            X_res, y_res = resampler.fit_resample(X_train.values, y_train.values)
        except ValueError as e:
            print(f"   ⚠️ {nombre_resampler} no generó muestras sintéticas: {e}")
            print(f"      → Se omite guardar {nombre_resampler}_{nombre_dataset}_*.csv")
            continue

        # ----------------------------
        # Cantidad de sintéticos
        # ----------------------------
        n_res = len(X_res)
        cant_sinteticos = n_res - n_original

        # Archivo con patrón nuevo
        nombre_archivo = (
            f"{nombre_resampler}_{nombre_dataset}"
            f"_I{percentil_isolation_etiqueta}"
            f"_sg{cant_sinteticos}_train.csv"
        )
        path_salida = os.path.join(ruta_clasicos, nombre_archivo)

        # Evitar sobrescritura si no corresponde
        if not overwrite and os.path.exists(path_salida):
            print(f"   ⚪ Omitido ({nombre_resampler}), ya existe: {nombre_archivo}")
            continue

        # ----------------------------
        # Guardar CSV
        # ----------------------------
        df_res = pd.DataFrame(X_res, columns=columnas)
        df_res[col_target] = y_res

        df_res.to_csv(path_salida, index=False)
        print(f"   ✅ Guardado: {path_salida} (sg={cant_sinteticos})")


In [15]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

def dibujarDistribuciones4_por_clase(
    listas_entropias,
    listas_mascara_entropia_baja,
    listas_mascara_vecino_minoritario,
    listas_mascara_pureza,
    listas_densidades,
    listas_riesgos,
    etiquetas_clases,
    nombre_dataset,
    etiqueta_configuracion,
    percentil_entropia=None,
    percentil_densidad=None,
    percentil_riesgo=None,
    carpeta_salida="../datasets/datasets_aumentados/logs/pcsmote/distribuciones",
):
    """
    Figura por dataset:
      filas   = nº de clases OVA
      columnas= [máscaras binarias, hist entropía, densidad, riesgo]
    """

    numero_clases = len(etiquetas_clases)

    # ========================
    # DEBUG INICIAL
    # ========================
    print("\n[DEBUG GRAFICADOR PCSMOTE POR CLASE]")
    print(f"  Dataset: {nombre_dataset}")
    print(f"  Config : {etiqueta_configuracion}")
    print(f"  Nº clases: {numero_clases}")

    if not (
        len(listas_entropias) == len(listas_mascara_entropia_baja)
        == len(listas_mascara_vecino_minoritario)
        == len(listas_mascara_pureza)
        == len(listas_densidades)
        == len(listas_riesgos)
        == numero_clases
    ):
        raise ValueError("Todas las listas deben tener la misma cantidad de clases.")

    for i, etiqueta in enumerate(etiquetas_clases):
        print(
            f"  clase {etiqueta}: "
            f"len(ent)={len(listas_entropias[i])}, "
            f"len(mask_ent)={len(listas_mascara_entropia_baja[i])}, "
            f"len(mask_vec)={len(listas_mascara_vecino_minoritario[i])}, "
            f"len(mask_pur)={len(listas_mascara_pureza[i])}, "
            f"len(dens)={len(listas_densidades[i])}, "
            f"len(riesgo)={len(listas_riesgos[i])}"
        )

    # ========================
    # SETUP FIGURA
    # ========================
    carpeta = Path(carpeta_salida)
    carpeta.mkdir(parents=True, exist_ok=True)

    fig, axes = plt.subplots(
        numero_clases,
        4,
        figsize=(22, 3.8 * numero_clases),
        squeeze=False,
    )

    # Dejamos buen margen izquierdo y derecho global,
    # y algo de espacio vertical/horizontal entre subplots
    fig.subplots_adjust(left=0.12, right=0.13, wspace=0.5, hspace=0.12)

    # ========================
    # LOOP POR CLASE
    # ========================
    for i in range(numero_clases):
        etiqueta_clase = etiquetas_clases[i]

        # --- vectores numéricos ---
        ent = np.asarray(listas_entropias[i], dtype=float)
        ent = ent[~np.isnan(ent)]

        dens = np.asarray(listas_densidades[i], dtype=float)
        dens = dens[~np.isnan(dens)]

        rie = np.asarray(listas_riesgos[i], dtype=float)
        rie = rie[~np.isnan(rie)]

        # --- máscaras binarias ---
        mask_ent = np.asarray(listas_mascara_entropia_baja[i], dtype=float)
        mask_vec = np.asarray(listas_mascara_vecino_minoritario[i], dtype=float)
        mask_pur = np.asarray(listas_mascara_pureza[i], dtype=float)

        c_ent = int(np.sum(mask_ent == 1))
        c_vec = int(np.sum(mask_vec == 1))
        c_pur = int(np.sum(mask_pur == 1))

        # =====================================================
        # Columna 0: MÁSCARAS BINARIAS (barras 0/1 con f=...)
        # =====================================================
        ax0 = axes[i, 0]

        x = np.array([0, 1, 2])
        cantidades = [c_ent, c_vec, c_pur]
        etiquetas_barras = ["Entropía baja", "Vecino min.", "Pureza"]
        colores_barras = ["C0", "green", "orange"]

        max_f = max(cantidades) if cantidades else 0
        offset = max_f * 0.06 if max_f > 0 else 0.5

        for j in range(3):
            ax0.bar(
                x[j],
                cantidades[j],
                width=0.6,
                alpha=0.8,
                edgecolor="black",
                color=colores_barras[j],
            )
            ax0.text(
                x[j],
                cantidades[j] + offset,
                f"f={cantidades[j]}",
                ha="center",
                va="bottom",
                fontsize=9,
            )

        ax0.set_xticks(x)
        ax0.set_xticklabels(etiquetas_barras, rotation=0)
        ax0.set_ylabel("Frecuencia")
        ax0.set_title("máscaras (binario)", loc="center", y=1.08)

        # Leyenda a la derecha, sin pisar el siguiente subplot
        box0 = ax0.get_position()
        ax0.set_position([box0.x0, box0.y0, box0.width * 0.8, box0.height])
        ax0.legend(
            ["Entropía baja", "Vecino minoritario", "Pureza (AND)"],
            loc="center left",
            bbox_to_anchor=(1.02, 0.5),
            borderaxespad=0.0,
            fontsize=8,
        )

        # =====================================================
        # Columna 1: ENTROPÍA (histograma + percentil)
        # =====================================================
        ax1 = axes[i, 1]

        if ent.size > 0:
            counts_e, bins_e, _ = ax1.hist(
                ent,
                bins=30,
                density=False,
                alpha=0.5,
                edgecolor="black",
                label="Entropía",
            )

            if percentil_entropia is not None:
                umbral_e = float(np.percentile(ent, percentil_entropia))
                ax1.axvline(
                    umbral_e,
                    color="red",
                    linewidth=2,
                    label=f"P{percentil_entropia:.1f} = {umbral_e:.4f}",
                )

                # f del bin donde cae el percentil
                idx_bin = np.digitize([umbral_e], bins_e) - 1
                idx_bin = int(np.clip(idx_bin[0], 0, len(counts_e) - 1))
                f_bin = int(counts_e[idx_bin])

                ax1.text(
                    umbral_e,
                    f_bin + max(counts_e) * 0.06,
                    f"f={f_bin}",
                    ha="center",
                    va="bottom",
                    fontsize=9,
                )
        else:
            ax1.text(0.5, 0.5, "Sin datos", ha="center", va="center")

        ax1.set_title("entropía", loc="center", y=1.08)
        ax1.set_xlabel("entropía")
        ax1.set_ylabel("Frecuencia")

        box1 = ax1.get_position()
        ax1.set_position([box1.x0, box1.y0, box1.width * 0.8, box1.height])
        ax1.legend(
            loc="center left",
            bbox_to_anchor=(1.02, 0.5),
            borderaxespad=0.0,
            fontsize=8,
        )

        # =====================================================
        # Columna 2: DENSIDAD
        # =====================================================
        ax2 = axes[i, 2]

        if dens.size > 0:
            counts_d, _, _ = ax2.hist(
                dens,
                bins=30,
                density=False,
                alpha=0.5,
                edgecolor="black",
                label="Densidad",
            )

            if percentil_densidad is not None:
                umbral_d = float(np.percentile(dens, percentil_densidad))
                ax2.axvline(
                    umbral_d,
                    color="red",
                    linewidth=2,
                    label=f"P{percentil_densidad:.1f} = {umbral_d:.4f}",
                )
        else:
            ax2.text(0.5, 0.5, "Sin datos", ha="center", va="center")

        ax2.set_title("densidad", loc="center", y=1.08)
        ax2.set_xlabel("densidad")
        ax2.set_ylabel("Frecuencia")

        box2 = ax2.get_position()
        ax2.set_position([box2.x0, box2.y0, box2.width * 0.8, box2.height])
        ax2.legend(
            loc="center left",
            bbox_to_anchor=(1.02, 0.5),
            borderaxespad=0.0,
            fontsize=8,
        )

        # =====================================================
        # Columna 3: RIESGO
        # =====================================================
        ax3 = axes[i, 3]

        if rie.size > 0:
            counts_r, _, _ = ax3.hist(
                rie,
                bins=30,
                density=False,
                alpha=0.5,
                edgecolor="black",
                label="Riesgo",
            )

            if percentil_riesgo is not None:
                umbral_r = float(np.percentile(rie, percentil_riesgo))
                ax3.axvline(
                    umbral_r,
                    color="red",
                    linewidth=2,
                    label=f"P{percentil_riesgo:.1f} = {umbral_r:.4f}",
                )
        else:
            ax3.text(0.5, 0.5, "Sin datos", ha="center", va="center")

        ax3.set_title("riesgo", loc="center", y=1.08)
        ax3.set_xlabel("riesgo")
        ax3.set_ylabel("Frecuencia")

        box3 = ax3.get_position()
        ax3.set_position([box3.x0, box3.y0, box3.width * 0.8, box3.height])
        ax3.legend(
            loc="center left",
            bbox_to_anchor=(1.02, 0.5),
            borderaxespad=0.0,
            fontsize=8,
        )

        # ===== título de fila: clase =====
        axes[i, 0].text(
            -0.22,
            0.5,
            f"clase {etiqueta_clase}",
            transform=axes[i, 0].transAxes,
            rotation=90,
            va="center",
            ha="right",
            fontsize=11,
            fontweight="bold",
        )

    # ========================
    # TÍTULO Y GUARDADO
    # ========================
    fig.suptitle(f"{nombre_dataset} | {etiqueta_configuracion}", fontsize=14)
    fig.tight_layout(rect=[0, 0, 1, 0.94])

    fname = f"distrib_4x_clases_{nombre_dataset}_{etiqueta_configuracion}.png"
    ruta_salida = carpeta / fname
    fig.savefig(ruta_salida, dpi=150)
    plt.close(fig)

    print(f"🖼  Distribuciones por CLASE guardadas en: {ruta_salida}")


### 🧬 Aumento de Datasets mediante Técnicas de Sobremuestreo

En esta etapa se genera una versión balanceada de cada dataset original mediante la aplicación de técnicas de sobremuestreo, con el objetivo de mitigar el desbalance de clases antes del entrenamiento de los modelos.

Actualmente, se emplea la técnica:

- `PCSMOTE` (Percentile-Controlled SMOTE), que permite controlar la generación de muestras sintéticas en función de percentiles de densidad, riesgo y pureza.

Para cada dataset, se exploran combinaciones específicas de parámetros según la técnica utilizada. Los datasets resultantes se almacenan en el directorio `datasets/datasets_aumentados/`, utilizando nombres de archivo que reflejan la configuración empleada (por ejemplo: `pcsmote_nombre_D25_R50_Pentropia_train.csv`).

> ⚠️ Esta fase no incluye entrenamiento ni validación de modelos. Su único propósito es generar conjuntos de datos aumentados a partir del conjunto de entrenamiento. La partición `train/test` se realiza previamente, y **solo la parte de entrenamiento es sometida a sobremuestreo**. El conjunto de prueba permanece sin modificar para garantizar una evaluación imparcial posterior.


In [16]:
# -------------------------
# DENSIDAD (radio por percentil) y umbral de densidad
# -------------------------
PERCENTILES_RADIO_DENSIDAD = [70, 85]     # PRD
UMBRALES_DENSIDAD = [0.50, 0.60]          # UD

# -------------------------
# RIESGO (radio por percentil) y umbral de riesgo
# -------------------------
PERCENTILES_RADIO_RIESGO = [30, 40, 50]   # PR
UMBRALES_RIESGO = [0.40, 0.50]            # UR

# -------------------------
# PUREZA
# -------------------------
# principal: proporción
UMBRALES_PROPORCION = [0.60, 0.70]        # Upp

# secundaria: entropía (solo para contraste metodológico)
PERCENTILES_ENTROPIA = [70]               # PE

# -------------------------
# Isolation
# -------------------------
PERCENTILES_ISOLATION = [0, 1, 3]         # I

percentiles_radio_densidad = []
percentiles_riesgo = []
criterios_pureza = []
umbrales_densidad = []
percentiles_entropia = []
umbrales_pureza_proporcion = []
umbral_riesgo = []
percentiles_isolation = []


# -------------------------
# FAMILIA 1: PUREZA = PROPORCION (PRINCIPAL)
# -------------------------
for prd in PERCENTILES_RADIO_DENSIDAD:
    for ud in UMBRALES_DENSIDAD:
        for pr in PERCENTILES_RADIO_RIESGO:
            for ur in UMBRALES_RIESGO:
                for upp in UMBRALES_PROPORCION:
                    for iso in PERCENTILES_ISOLATION:

                        percentiles_radio_densidad.append(prd)
                        percentiles_riesgo.append(pr)
                        criterios_pureza.append("proporcion")
                        umbrales_densidad.append(ud)
                        percentiles_entropia.append(None)
                        umbrales_pureza_proporcion.append(upp)
                        umbral_riesgo.append(ur)
                        percentiles_isolation.append(iso)


# -------------------------
# FAMILIA 2: PUREZA = ENTROPIA (SECUNDARIA)
# (solo contraste, muy acotada)
# -------------------------
for prd in PERCENTILES_RADIO_DENSIDAD:
    for ud in UMBRALES_DENSIDAD:
        for pr in [40]:               # fijo para no inflar el espacio
            for ur in [0.45]:         # fijo (punto medio)
                for pe in PERCENTILES_ENTROPIA:
                    for iso in [0, 1]:  # acotado

                        percentiles_radio_densidad.append(prd)
                        percentiles_riesgo.append(pr)
                        criterios_pureza.append("entropia")
                        umbrales_densidad.append(ud)
                        percentiles_entropia.append(pe)
                        umbrales_pureza_proporcion.append(None)
                        umbral_riesgo.append(ur)
                        percentiles_isolation.append(iso)


In [17]:
import time

def actualizar_eta(
    idx_actual,
    total,
    duracion_iteracion,
    estado_eta
):
    """
    Actualiza un promedio móvil de duración por iteración y estima ETA.

    Parámetros
    ----------
    idx_actual : int
        Iteración actual (1-based).
    total : int
        Total de iteraciones.
    duracion_iteracion : float
        Tiempo (en segundos) que tardó la iteración actual.
    estado_eta : dict
        Estado mutable con:
            - 'promedio_movil'
            - 'alpha'

    Retorna
    -------
    dict con:
        - promedio_movil
        - eta_segundos
        - restante
    """

    alpha = estado_eta["alpha"]

    if estado_eta["promedio_movil"] is None:
        promedio_movil = duracion_iteracion
    else:
        promedio_movil = (
            alpha * duracion_iteracion
            + (1.0 - alpha) * estado_eta["promedio_movil"]
        )

    estado_eta["promedio_movil"] = promedio_movil

    restantes = total - idx_actual
    eta_segundos = restantes * promedio_movil

    return {
        "promedio_movil": promedio_movil,
        "eta_segundos": eta_segundos,
        "restantes": restantes,
    }
    

In [18]:
import time
from pathlib import Path

combinaciones = list(zip(
    percentiles_radio_densidad,
    percentiles_riesgo,
    criterios_pureza,
    umbrales_densidad,
    percentiles_entropia,
    umbrales_pureza_proporcion,
    umbral_riesgo,
    percentiles_isolation,
))

datasets_a_ignorar = {
    "shuttle",
    "iris",
    # "glass",
    # "heart",
    # "wdbc",
    # "ecoli",
    # "us_crime",
    # "predict_faults",
    "gear_vibration"
    "telco_churn"
}

# Asumimos que solo usás [0, 1, 3]
ISOS_VALIDOS = {0, 1, 3}

def _hms_desde_segundos(segundos):
    segundos = int(segundos)
    h = segundos // 3600
    m = (segundos % 3600) // 60
    s = segundos % 60
    return f"{h:02d}:{m:02d}:{s:02d}"

# ✅ instancia de Utils (si ya tenés una global, usá esa)
utils = Utils()

for nombre_dataset, config in config_datasets.items():
    if nombre_dataset in datasets_a_ignorar:
        continue

    print(f"\n📁 Dataset: {nombre_dataset}")

    total_configs = len(combinaciones)

    # ✅ ETA separada por isolation (I=0,1,3)
    estado_eta_por_iso = {
        0: {"promedio_movil": None, "alpha": 0.20},
        1: {"promedio_movil": None, "alpha": 0.20},
        3: {"promedio_movil": None, "alpha": 0.20},
    }

    # ✅ acumuladores por iso: promedios por bloque de 10 PARA CADA ISO
    acumulado_segundos_por_iso = {0: 0.0, 1: 0.0, 3: 0.0}
    acumulado_iters_por_iso = {0: 0, 1: 0, 3: 0}

    # ✅ contadores de cuántas configs totales hay por iso (para ponderar ETA)
    total_por_iso = {0: 0, 1: 0, 3: 0}
    for (_, _, _, _, _, _, _, iso) in combinaciones:
        iso_int = int(iso)
        if iso_int in total_por_iso:
            total_por_iso[iso_int] += 1

    # ✅ contadores de progreso por iso (cuántas ya se hicieron por iso)
    hechas_por_iso = {0: 0, 1: 0, 3: 0}

    # =======================================================
    # ✅ LOG POR MUESTRA: buffer externo -> CSV -> XLSX al final
    # =======================================================
    base_logs = Path("../datasets/datasets_aumentados/logs/pcsmote/por_muestras")
    base_logs.mkdir(parents=True, exist_ok=True)

    log_path_csv  = base_logs / Utils.safe_token(f"log_pcsmote_x_muestra_{nombre_dataset}.csv")
    log_path_xlsx = base_logs / Utils.safe_token(f"log_pcsmote_x_muestra_{nombre_dataset}.xlsx")

    # ✅ corrida limpia de logs
    utils.borrar_archivo_log(log_path_csv)
    utils.borrar_archivo_log(log_path_xlsx)

    # ✅ buffer externo (cero pérdida). Guardamos dicts de filas.
    buffer_logs_por_muestra = []
    TAM_BLOQUE_LOG = 10  # cada 10 configuraciones volcamos a CSV

    # (opcional) contador de configuraciones con sampler válido (para flush por bloque)
    configs_desde_ultimo_flush = 0

    for idx, (prdens, priesgo, criterio, udens, pentropia, upproporcion, uriesgo, p_isol) in enumerate(combinaciones, start=1):

        iso_int = int(p_isol)
        if iso_int not in ISOS_VALIDOS:
            raise ValueError(f"Se esperaba percentil_isolation en {sorted(ISOS_VALIDOS)}, pero llegó: {p_isol}")

        porcentaje = (idx / total_configs) * 100.0
        print(
            f"\n   ▶ Ejecución ({idx}/{total_configs}) "
            f"[{porcentaje:6.2f}%] | "
            f"D={prdens} | R={priesgo} | P={criterio} | "
            f"udensidad={udens} | pentropia={pentropia} | "
            f"uprop={upproporcion} | I={iso_int}"
        )

        t_ini = time.perf_counter()

        try:
            # Caso base para este percentil de IsolationForest
            base_train, base_test, X_test_base, y_test_base, idx_train = generar_caso_base(
                nombre_dataset,
                config,
                porcentaje_limpieza=iso_int,
            )
            print(f"🟦 Caso base generado (I{iso_int}):\n - Train: {base_train}\n - Test: {base_test}")

            col_target = config.get("col_target", "target")
            df_base_train = pd.read_csv(base_train)

            if col_target not in df_base_train.columns:
                raise ValueError(f"La columna target '{col_target}' no está en {base_train}")

            X_train_df = df_base_train.drop(columns=[col_target])
            y_train_series = df_base_train[col_target]

            X_train_base = X_train_df.values
            y_train_base = y_train_series.values

            # Clásicos
            generar_aumentaciones_clasicas_y_guardar(
                nombre_dataset=nombre_dataset,
                X_train=X_train_df,
                y_train=y_train_series,
                col_target=col_target,
                ruta_clasicos=RUTA_CLASICOS,
                overwrite=False,
                percentil_isolation_etiqueta=iso_int,
            )

            # PCSMOTE
            df_header, df_log, sampler = aumentar_dataset_pcsmote_y_guardar(
                nombre_dataset               = nombre_dataset,
                X_train_base                 = X_train_base,
                y_train_base                 = y_train_base,

                percentil_radio_densidad     = prdens,
                percentil_entropia           = pentropia,
                percentil_riesgo             = priesgo,
                criterio_pureza              = criterio,
                umbral_densidad              = udens,
                umbral_pureza_proporcion     = upproporcion,
                umbral_riesgo                = uriesgo,
                percentil_isolation_etiqueta = iso_int,

                col_target                   = col_target,
                X_test_base                  = X_test_base,
                y_test_base                  = y_test_base,
                idx_train                    = idx_train,
            )

            if sampler is None:
                print("❌ Falló la generación con PCSMOTE.")
                continue

            # =======================================================
            # ✅ BUFFER EXTERNO: recolectar filas desde sampler.logs_por_muestra
            # =======================================================
            logs_iteracion = getattr(sampler, "logs_por_muestra", None)
            if logs_iteracion:
                # logs_por_muestra es lista de dicts (según tu clase)
                for fila in logs_iteracion:
                    buffer_logs_por_muestra.append(fila)

            configs_desde_ultimo_flush += 1

            # ✅ FLUSH cada 10 configs (append a CSV, rápido)
            if configs_desde_ultimo_flush >= TAM_BLOQUE_LOG:
                if buffer_logs_por_muestra:
                    # volcamos buffer a CSV usando el método del sampler (reutiliza tu estilo)
                    sampler.logs_por_muestra = buffer_logs_por_muestra
                    sampler.exportar_log_muestras_csv(
                        ruta_csv=log_path_csv,
                        append=True,
                    )

                    # limpiar buffer externo + limpiar logs internos por prolijidad
                    buffer_logs_por_muestra = []
                    sampler.limpiar_logs_por_muestra()

                    print(f"📄 Log por MUESTRA volcado a CSV (bloque):\n    {log_path_csv}\n")

                configs_desde_ultimo_flush = 0

        finally:
            t_fin = time.perf_counter()
            duracion = t_fin - t_ini

            # ✅ progreso por iso
            hechas_por_iso[iso_int] += 1

            # ✅ acumular bloque por iso
            acumulado_segundos_por_iso[iso_int] += duracion
            acumulado_iters_por_iso[iso_int] += 1

            # ✅ cada 10 ITERACIONES DE ESE ISO: actualizar su ETA
            if acumulado_iters_por_iso[iso_int] >= 10:
                prom_bloque_iso = acumulado_segundos_por_iso[iso_int] / acumulado_iters_por_iso[iso_int]

                actualizar_eta(
                    idx_actual=hechas_por_iso[iso_int],
                    total=total_por_iso[iso_int],
                    duracion_iteracion=prom_bloque_iso,
                    estado_eta=estado_eta_por_iso[iso_int],
                )

                # reset bloque por iso
                acumulado_segundos_por_iso[iso_int] = 0.0
                acumulado_iters_por_iso[iso_int] = 0

            # ✅ imprimir ETA global ponderada cada 10 configs globales
            if idx % 10 == 0:
                eta_total = 0.0
                detalle = []

                for iso_k in [0, 1, 3]:
                    promedio_movil = estado_eta_por_iso[iso_k]["promedio_movil"]

                    restantes_iso = total_por_iso[iso_k] - hechas_por_iso[iso_k]
                    if restantes_iso < 0:
                        restantes_iso = 0

                    if promedio_movil is None:
                        detalle.append(f"I{iso_k}: sin base | rest={restantes_iso}")
                        continue

                    eta_iso = restantes_iso * promedio_movil
                    eta_total += eta_iso
                    detalle.append(f"I{iso_k}: {promedio_movil:.2f}s | rest={restantes_iso}")

                print(
                    "   ⏱️ ETA ponderada por I | "
                    f"ETA aprox: {_hms_desde_segundos(eta_total)} | "
                    f"{idx}/{total_configs} | "
                    + " | ".join(detalle)
                )

    # =======================================================
    # ✅ FLUSH FINAL (cero pérdida): volcar lo que quede en buffer
    # =======================================================
    if buffer_logs_por_muestra:
        # creamos un objeto "contenedor" para reutilizar el método de export
        # Si tu exportar_log_muestras_csv está en sampler y no en utils, necesitamos un sampler.
        # Como esta rama es final, si el último sampler no existió, usamos un UtilsLogDummy mínimo.

        try:
            # Si existe algún sampler en scope y tiene el método, lo usamos
            if "sampler" in locals() and sampler is not None and hasattr(sampler, "exportar_log_muestras_csv"):
                sampler.logs_por_muestra = buffer_logs_por_muestra
                sampler.exportar_log_muestras_csv(ruta_csv=log_path_csv, append=True)
                sampler.limpiar_logs_por_muestra()
            else:
                # fallback: usar Utils para exportar desde un DataFrame (sin perder datos)
                df_flush = pd.DataFrame(buffer_logs_por_muestra)
                existe = log_path_csv.exists()
                df_flush.to_csv(
                    log_path_csv,
                    mode="a",
                    index=False,
                    header=(not existe),
                    encoding="utf-8",
                )

            print(f"📄 Log por MUESTRA volcado a CSV (final):\n    {log_path_csv}\n")

        finally:
            buffer_logs_por_muestra = []

    # =======================================================
    # ✅ CONVERSIÓN FINAL: CSV -> XLSX (una vez)
    # =======================================================
    utils.convertir_csv_a_excel(
        ruta_csv=log_path_csv,
        ruta_excel=log_path_xlsx,
    )
    print(f"✅ XLSX final generado:\n    {log_path_xlsx}\n")



📁 Dataset: us_crime

   ▶ Ejecución (1/152) [  0.66%] | D=70 | R=30 | P=proporcion | udensidad=0.5 | pentropia=None | uprop=0.6 | I=0
[us_crime] Split: X_train=(1595, 100), X_test=(399, 100)
✅ Caso base generado para us_crime
   Train: ../datasets/datasets_aumentados/base/us_crime_I0_tm1595_train.csv  (rows=1595)
   Test : ../datasets/datasets_aumentados/base/us_crime_tdataset1994_tm399_test.csv   (rows=399)
🟦 Caso base generado (I0):
 - Train: ../datasets/datasets_aumentados/base/us_crime_I0_tm1595_train.csv
 - Test: ../datasets/datasets_aumentados/base/us_crime_tdataset1994_tm399_test.csv
🔧 Aumentación clásica (en memoria) para: us_crime
   X_train shape: (1595, 100), y_train shape: (1595,)
   🔁 Aplicando smote ...
   ✅ Guardado: ..\datasets\datasets_aumentados\resampler_clasicos\smote_us_crime_I0_sg1355_train.csv (sg=1355)
   🔁 Aplicando borderlinesmote ...
   ✅ Guardado: ..\datasets\datasets_aumentados\resampler_clasicos\borderlinesmote_us_crime_I0_sg1355_train.csv (sg=1355)
   🔁 

KeyboardInterrupt: 